In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
conv = {
    'HR': 'Heavy Rail',
    'CB': 'Commuter Bus',
    'MB': 'Bus',
    'DR': 'Demand Response',
    'RB': 'Bus Rapid Transit',
    'LR': 'Light Rail',
    'VP': 'Vanpool',
    'YR': 'Hybrid Rail',
    'CR': 'Commuter Rail',
    'DT': 'Demand Response Taxi',
    'FB': 'Ferryboat',
    'TB': 'Trolleybus',
    'SR': 'Streetcar Rail',
    'PB': 'Publico',
    'MG': 'Monorail/Automated Guideway',
    'CC': 'Cable Car',
    'IP': 'Inclined Plane',
    'TR': 'Aerial Tramway',
    'AR': 'Alaska Railroad',
    'nan': 'nan',
    'UZA': 'Urbanized Area',
    'JT': 'Jitney',
    'Q': 'Questionable',
    'W': 'Waived'
}


def drop_cols(col, df):
    df_len = len(df)
    df = df.dropna(subset=[col])
    print(f'Dropped {df_len - len(df)} rows for having null {col}')
    return df

In [ ]:
# Perform additional cleaning steps
df = pd.read_csv('../data/clean_metrics_18_22.csv')
df['mode_name'] = df['mode'].apply(lambda x: conv[x] if x in conv else x)
df = drop_cols('city', df)
df = drop_cols('passenger_miles', df)
df = drop_cols('cost_per_passenger_mile', df)

In [ ]:
# Check Modesto Commuter bus
display(df[(df.city == 'Modesto') & (df.mode_name == 'Commuter Bus')] \
    .loc[:, ['city', 'year', 'cost_per_passenger_mile']] )

# Drop Modesto Commuter bus
df = df[(df.city != 'Modesto') | (df.mode_name != 'Commuter Bus')]